In [2]:
import pandas as pd

In [3]:
# Load dataset
df = pd.read_excel('../data/Online Retail.xlsx')


In [4]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Basic cleaning
df = df[df['InvoiceNo'].notna()]
df = df[df['CustomerID'].notna()]
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]


In [6]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Create TotalAmount = Quantity x UnitPrice
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# Keep only relevant columns
df = df[['InvoiceNo', 'InvoiceDate', 'CustomerID', 'Country', 'Quantity', 'UnitPrice', 'TotalAmount']]

# Save cleaned data
df.to_csv('../data/processed.csv', index=False)

df.head()

,InvoiceNo,InvoiceDate,CustomerID,Country,Quantity,UnitPrice,TotalAmount
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,6,2.55,15.30
1,536365,2010-12-01 08:26:00,17850.0,United Kingdom,6,3.39,20.34
2,536365,2010-12-01 08:26:00,17850.0,United Kingdom,8,2.75,22.00
3,536365,2010-12-01 08:26:00,17850.0,United Kingdom,6,3.39,20.34
4,536365,2010-12-01 08:26:00,17850.0,United Kingdom,6,3.39,20.34


In [7]:
import pandas as pd
from datetime import timedelta

# Load cleaned data
df = pd.read_csv('../data/processed.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Snapshot date (latest purchase in data + 1 day)
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)


In [8]:
# Group by customer
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm.head()


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [9]:
# Set cutoff dates
cutoff_date = df['InvoiceDate'].min() + pd.DateOffset(months=6)
train_df = df[df['InvoiceDate'] <= cutoff_date]
test_df = df[df['InvoiceDate'] > cutoff_date]

# RFM from training window
rfm_train = train_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (cutoff_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).reset_index()
rfm_train.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Target: next 3-month spending
clv_target = test_df.groupby('CustomerID')['TotalAmount'].sum().reset_index()
clv_target.columns = ['CustomerID', 'Future_Spend']

# Merge
data = pd.merge(rfm_train, clv_target, on='CustomerID', how='inner')
data.head()


,CustomerID,Recency,Frequency,Monetary,Future_Spend
0,12347.0,54,3,1823.43,2486.57
1,12348.0,56,3,1487.24,310.00
2,12352.0,70,5,1561.81,944.23
3,12356.0,53,2,2753.08,58.35
4,12359.0,113,2,2386.41,3986.17


In [10]:
data.to_csv('../data/model_data.csv', index=False)


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load RFM + CLV data
data = pd.read_csv('../data/model_data.csv')

# Define features and target
X = data[['Recency', 'Frequency', 'Monetary']]
y = data['Future_Spend']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

# Evaluate
print("Linear Regression:")
print("MSE:", mean_squared_error(y_test, y_pred_lr))
print("R2 Score:", r2_score(y_test, y_pred_lr))


Linear Regression:
MSE: 7289938.434904299
R2 Score: 0.20803192026066486


In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
print("XGBoost Regressor:")
print("MSE:", mean_squared_error(y_test, y_pred_xgb))
print("R2 Score:", r2_score(y_test, y_pred_xgb))


XGBoost Regressor:
MSE: 7632115.431862164
R2 Score: 0.17085831973831012


NameError: name 'model' is not defined

In [17]:
import sys
!{sys.executable} -m pip install xgboost


   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/150.0 MB 6.7 MB/s eta 0:00:23
   ---------------------------------------- 1.8/150.0 MB 4.6 MB/s eta 0:00:33
    --------------------------------------- 2.6/150.0 MB 4.3 MB/s eta 0:00:35
    --------------------------------------- 3.4/150.0 MB 3.9 MB/s eta 0:00:38
   - -------------------------------------- 4.2/150.0 MB 4.1 MB/s eta 0:00:36
   - -------------------------------------- 5.2/150.0 MB 4.0 MB/s eta 0:00:36
   - -------------------------------------- 6.0/150.0 MB 4.0 MB/s eta 0:00:36
   - -------------------------------------- 6.8/150.0 MB 4.0 MB/s eta 0:00:36
   -- ------------------------------------- 7.6/150.0 MB 4.0 MB/s eta 0:00:36
   -- ------------------------------------- 8.4/150.0 MB 4.0 MB/s eta 0:00:36
   -- ------------------------------------- 9.2/150.0 MB 4.0 MB/s eta 0:00:36
   -- ------------------------------------- 10.0/150.0 MB 4.0 MB/s eta 

In [21]:
import joblib